# Iteration 3: organise parameters

## Plain English Summary

Before we add in results collection the organisation of the model is modified so that parameters are separated from model logic.  Given the "hidden code" issue in iteration 2 the input included a command to show all generator functions.

The same tests can be applied here as iteration 1, 2, and 3. These should now be easier as parameters should not be hardcoded and are instead accessible via the Experiment class.




## Notebook imports

In [2]:
from IPython.display import Markdown, display

import numpy as np
import math

## LLM Prompt 

In [4]:
display(Markdown("../01_perplexity/inputs/iteration_3.md"))

## Main command

Modify the code so that all model parameters are stored within a class called Experiment that is separate from the model's logic.  

The list of parameters include:

1. the means of the arrival distributions
2. the unplanned patient treatment time mean and standard deviations.
3. the elective patients mean treatment time.
4. The number of critical care beds available
5. The duration of intensive cleaning following a patient discharge.
6. simulation run length

The parameters should be passed to the Experiment class constructor method when it is created. please provide default values

Show all of the model code: this includes all patient generator functions in the CCModel class.

## Date and time of input to LLM

> 1925 04 June 2024 

## Observations on the LLM Output

* The `Experiment` class has been created.  All parameters are passed in the constructor method as separate parameters.
* The mean elective treatment time is included in `stay_distributions`, but there is also an `elective_treatment_mean` variable.  It is this variable that is used by `elective_surgery_process` to sample from the exponential distribution.
* There is a bug in the code. The `run` method uses the module level scope variable `env` instead of the instance variable `self.env` that `CCUModel` has been passed.  This was picked up by the tester.  The LLM was asked to fix the bug.

## LLM Output Stage 2

In this modified code:
The Experiment class stores all the model parameters as class attributes with default values.
The CCU class now takes an Experiment instance as an argument in its constructor.
All references to model parameters in the CCU class have been replaced with references to the corresponding attributes of the Experiment instance.
This separation of model parameters and logic makes it easier to modify parameters and run different experiments without changing the core model code.

In [5]:
import simpy
import numpy as np
from scipy.stats import lognorm

class Experiment:
    def __init__(self, accident_emergency_arrival_rate=22.72, wards_arrival_rate=26.0, emergency_surgery_arrival_rate=37.0,
                 other_hospitals_arrival_rate=47.2, xray_department_arrival_rate=575.0, elective_surgery_arrival_rate=17.91,
                 accident_emergency_lognormal_mu=128.79, accident_emergency_lognormal_sigma=267.51,
                 wards_lognormal_mu=177.89, wards_lognormal_sigma=276.54,
                 emergency_surgery_lognormal_mu=140.15, emergency_surgery_lognormal_sigma=218.02,
                 other_hospitals_lognormal_mu=212.86, other_hospitals_lognormal_sigma=457.67,
                 xray_department_lognormal_mu=87.53, xray_department_lognormal_sigma=108.67,
                 elective_surgery_treatment_rate=57.34, num_critical_care_beds=24,
                 intensive_cleaning_duration=5, run_length=12 * 30 * 24):
        self.accident_emergency_arrival_rate = accident_emergency_arrival_rate
        self.wards_arrival_rate = wards_arrival_rate
        self.emergency_surgery_arrival_rate = emergency_surgery_arrival_rate
        self.other_hospitals_arrival_rate = other_hospitals_arrival_rate
        self.xray_department_arrival_rate = xray_department_arrival_rate
        self.elective_surgery_arrival_rate = elective_surgery_arrival_rate
        self.accident_emergency_lognormal_mu = accident_emergency_lognormal_mu
        self.accident_emergency_lognormal_sigma = accident_emergency_lognormal_sigma
        self.wards_lognormal_mu = wards_lognormal_mu
        self.wards_lognormal_sigma = wards_lognormal_sigma
        self.emergency_surgery_lognormal_mu = emergency_surgery_lognormal_mu
        self.emergency_surgery_lognormal_sigma = emergency_surgery_lognormal_sigma
        self.other_hospitals_lognormal_mu = other_hospitals_lognormal_mu
        self.other_hospitals_lognormal_sigma = other_hospitals_lognormal_sigma
        self.xray_department_lognormal_mu = xray_department_lognormal_mu
        self.xray_department_lognormal_sigma = xray_department_lognormal_sigma
        self.elective_surgery_treatment_rate = elective_surgery_treatment_rate
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.run_length = run_length

class CCU:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_id_counter = 0
        self.cancelled_operations = 0

        # Set up random number generators for each patient type
        self.rng_accident_emergency = np.random.default_rng(seed=42)
        self.rng_wards = np.random.default_rng(seed=43)
        self.rng_emergency_surgery = np.random.default_rng(seed=44)
        self.rng_other_hospitals = np.random.default_rng(seed=45)
        self.rng_xray_department = np.random.default_rng(seed=46)
        self.rng_elective_surgery = np.random.default_rng(seed=47)

        # Set up critical care bed resource
        self.critical_care_beds = simpy.Resource(env, capacity=self.experiment.num_critical_care_beds)

    def lognormal_to_normal(self, mu, sigma):
        """Convert lognormal parameters to underlying normal distribution"""
        zeta = np.log(mu**2 / np.sqrt(sigma**2 + mu**2))
        sigma_norm = np.sqrt(np.log(sigma**2 / mu**2 + 1))
        mu_norm = zeta
        return mu_norm, sigma_norm

    def accident_emergency_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Patient {patient_id} arrived from Accident and Emergency at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_accident_emergency.exponential(self.experiment.accident_emergency_arrival_rate))
            self.env.process(self.unplanned_admissions_process(patient_id, "Accident and Emergency"))

    def wards_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Patient {patient_id} arrived from the Wards at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_wards.exponential(self.experiment.wards_arrival_rate))
            self.env.process(self.unplanned_admissions_process(patient_id, "Wards"))

    def emergency_surgery_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Patient {patient_id} arrived from Emergency Surgery at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_emergency_surgery.exponential(self.experiment.emergency_surgery_arrival_rate))
            self.env.process(self.unplanned_admissions_process(patient_id, "Emergency Surgery"))

    def other_hospitals_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Patient {patient_id} arrived from Other Hospitals at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_other_hospitals.exponential(self.experiment.other_hospitals_arrival_rate))
            self.env.process(self.unplanned_admissions_process(patient_id, "Other Hospitals"))

    def xray_department_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Patient {patient_id} arrived from the X-Ray Department at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_xray_department.exponential(self.experiment.xray_department_arrival_rate))
            self.env.process(self.unplanned_admissions_process(patient_id, "X-Ray Department"))

    def elective_surgery_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            print(f"Elective Patient {patient_id} arrived at {self.env.now:.2f} hours")
            yield self.env.timeout(self.rng_elective_surgery.normal(self.experiment.elective_surgery_arrival_rate, 3.16))
            self.env.process(self.elective_admissions_process(patient_id))

    def unplanned_admissions_process(self, patient_id, source):
        print(f"Patient {patient_id} from {source} requests a critical care bed at {self.env.now:.2f} hours")
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {patient_id} from {source} admitted to a critical care bed at {self.env.now:.2f} hours")

            # Determine length of stay based on source
            if source == "Accident and Emergency":
                mu, sigma = self.lognormal_to_normal(self.experiment.accident_emergency_lognormal_mu, self.experiment.accident_emergency_lognormal_sigma)
                length_of_stay = self.rng_accident_emergency.lognormal(mu, sigma)
            elif source == "Wards":
                mu, sigma = self.lognormal_to_normal(self.experiment.wards_lognormal_mu, self.experiment.wards_lognormal_sigma)
                length_of_stay = self.rng_wards.lognormal(mu, sigma)
            elif source == "Emergency Surgery":
                mu, sigma = self.lognormal_to_normal(self.experiment.emergency_surgery_lognormal_mu, self.experiment.emergency_surgery_lognormal_sigma)
                length_of_stay = self.rng_emergency_surgery.lognormal(mu, sigma)
            elif source == "Other Hospitals":
                mu, sigma = self.lognormal_to_normal(self.experiment.other_hospitals_lognormal_mu, self.experiment.other_hospitals_lognormal_sigma)
                length_of_stay = self.rng_other_hospitals.lognormal(mu, sigma)
            else:  # X-Ray Department
                mu, sigma = self.lognormal_to_normal(self.experiment.xray_department_lognormal_mu, self.experiment.xray_department_lognormal_sigma)
                length_of_stay = self.rng_xray_department.lognormal(mu, sigma)

            yield self.env.timeout(length_of_stay)
            print(f"Patient {patient_id} from {source} discharged from a critical care bed at {self.env.now:.2f} hours")

            # Intensive cleaning
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Intensive cleaning completed for Patient {patient_id} from {source} at {self.env.now:.2f} hours")

    def elective_admissions_process(self, patient_id):
        print(f"Elective Patient {patient_id} requests a critical care bed at {self.env.now:.2f} hours")
        if self.critical_care_beds.count == self.critical_care_beds.capacity:
            print(f"Elective Patient {patient_id} operation cancelled at {self.env.now:.2f} hours due to lack of available beds")
            self.cancelled_operations += 1
        else:
            with self.critical_care_beds.request() as req:
                yield req
                print(f"Elective Patient {patient_id} admitted to a critical care bed at {self.env.now:.2f} hours")
                length_of_stay = self.rng_elective_surgery.exponential(self.experiment.elective_surgery_treatment_rate)
                yield self.env.timeout(length_of_stay)
                print(f"Elective Patient {patient_id} discharged from a critical care bed at {self.env.now:.2f} hours")

                # Intensive cleaning
                yield self.env.timeout(self.experiment.intensive_cleaning_duration)
                print(f"Intensive cleaning completed for Elective Patient {patient_id} at {self.env.now:.2f} hours")

    def run(self):
        # Start patient arrival processes
        accident_emergency_process = self.env.process(self.accident_emergency_arrivals())
        wards_process = self.env.process(self.wards_arrivals())
        emergency_surgery_process = self.env.process(self.emergency_surgery_arrivals())
        other_hospitals_process = self.env.process(self.other_hospitals_arrivals())
        xray_department_process = self.env.process(self.xray_department_arrivals())
        elective_surgery_process = self.env.process(self.elective_surgery_arrivals())

        self.env.run(until=self.experiment.run_length)

        print(f"Total cancelled elective operations: {self.cancelled_operations}")

# Run the simulation
experiment = Experiment()
env = simpy.Environment()
ccu = CCU(env, experiment)
ccu.run()


Patient 0 arrived from Accident and Emergency at 0.00 hours
Patient 1 arrived from the Wards at 0.00 hours
Patient 2 arrived from Emergency Surgery at 0.00 hours
Patient 3 arrived from Other Hospitals at 0.00 hours
Patient 4 arrived from the X-Ray Department at 0.00 hours
Elective Patient 5 arrived at 0.00 hours
Patient 6 arrived from the Wards at 4.30 hours
Patient 1 from Wards requests a critical care bed at 4.30 hours
Patient 1 from Wards admitted to a critical care bed at 4.30 hours
Patient 7 arrived from the Wards at 6.54 hours
Patient 6 from Wards requests a critical care bed at 6.54 hours
Patient 6 from Wards admitted to a critical care bed at 6.54 hours
Patient 8 arrived from Emergency Surgery at 13.71 hours
Patient 2 from Emergency Surgery requests a critical care bed at 13.71 hours
Patient 2 from Emergency Surgery admitted to a critical care bed at 13.71 hours
Elective Patient 9 arrived at 15.80 hours
Elective Patient 5 requests a critical care bed at 15.80 hours
Elective Pat

TODO: Testing

Perplexity is having a bad evening. Additional prompts are undoing previous prompts. I want this: 
> Do not assign self.patient_id_counter to a patient_id variable in generators. Within admission processes, do not use patient_id as an argument, refer to it directly as self.patient_id_counter

Each time it removes the experiment class. Adding the experiment class back removes the previous changes.

## Testing

Here we do a number of regression tests to check that previous functionality is not affected by the `Experiment` class.

From iteration 0

* **Extreme value test 1**: Wards, Em Surgery, other hospitals, x-ray have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All unplanned patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Elective."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Elective."
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 240 simulated time periods
    * Actual result **(PASS)**: The model runs no longer than 240 simulated time periods
* **Extreme value test 3**: `critical_care_beds` = 1
    * Expected result: queues form after first arrival.
    * Actual result **(PASS)**: queues form after first arrival. 
* **Extreme value test 4**: means of stay_distributions set to $M$ a very large number.
    * Expected result: after 24 arrivals queues form and no patients are admitted.
    * Actual result **(PASS)**: after 24 arrivals queues form and no patients are admitted.


### Test 1: Extreme value test 1

PASS

In [6]:
M = 10_000_000

In [7]:
def run_test(experiment):
    # Create a SimPy environment
    env = simpy.Environment()
    ccu_model = CCUModel(env, experiment)
    ccu_model.run()
    return ccu_model

In [8]:
def extreme_value_test_1(extreme_value=M):
    '''
    Extreme value test 1: 
    
    Wards, Em Surgery, other hospitals, x-ray, electives have their inter-arrival time
    set to $M$ a very large number
    Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    '''
    mod_interarrival_means = [22.72] + [extreme_value] * 5 
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    run_test(experiment)

In [9]:
extreme_value_test_1()

Patient 1 arrived from Accident and Emergency at time 10.922915837439156
Patient 1 admitted to critical care bed at time 10.922915837439156
Patient 2 arrived from Accident and Emergency at time 56.83516219821753
Patient 2 admitted to critical care bed at time 56.83516219821753
Patient 3 arrived from Accident and Emergency at time 73.9034076509651
Patient 3 admitted to critical care bed at time 73.9034076509651
Patient 4 arrived from Accident and Emergency at time 74.16882762713699
Patient 4 admitted to critical care bed at time 74.16882762713699
Patient 4 discharged from critical care bed at time 77.62318317670336
Critical care bed is available for next patient at time 82.62318317670336
Patient 4 discharged from critical care bed at time 109.21779435000593
Patient 5 arrived from Accident and Emergency at time 109.8014728202405
Patient 5 admitted to critical care bed at time 109.8014728202405
Patient 5 discharged from critical care bed at time 109.93245655348137
Critical care bed is ava

### Test 2: Extreme value test 2

PASS

In [10]:
def extreme_value_test_2(extreme_value=M):
    '''
    Extreme value test 2: 
    
    All unplanned admissions have their inter-arrival time
    set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model is "Elective"
    '''
    mod_interarrival_means=[extreme_value] * 5 + [17.91]
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    run_test(experiment)

In [11]:
M = 10_000_000
extreme_value_test_2()

Elective surgery patient 1 arrived at time 16.91683024877911
Elective surgery patient 1 admitted to critical care bed at time 16.91683024877911
Elective surgery patient 2 arrived at time 34.51856335894503
Elective surgery patient 2 admitted to critical care bed at time 34.51856335894503
Elective surgery patient 3 arrived at time 53.688691772316766
Elective surgery patient 3 admitted to critical care bed at time 53.688691772316766
Elective surgery patient 3 discharged from critical care bed at time 65.05459160515842
Elective surgery patient 3 discharged from critical care bed at time 66.23276097219528
Critical care bed is available for next patient at time 70.05459160515842
Elective surgery patient 4 arrived at time 71.12101795928653
Elective surgery patient 4 admitted to critical care bed at time 71.12101795928653
Critical care bed is available for next patient at time 71.23276097219528
Elective surgery patient 5 arrived at time 90.78047863991497
Elective surgery patient 5 admitted to 

### Test 3: Different run length

PASS

In [12]:
def test_run_length(new_run_length):
    '''
    Different run length

    Expected result Run length should be no longer than `new_run_length`
    '''
    experiment = Experiment(run_length=new_run_length)
    model = run_test(experiment)
    return model.env.now

In [13]:
test_run_length(10)

Patient 1 arrived from other hospitals at time 3.071735971407217
Patient 1 admitted to critical care bed at time 3.071735971407217
Patient 2 arrived from other hospitals at time 8.990331665638939
Patient 2 admitted to critical care bed at time 8.990331665638939


10

### Test 4: Extreme value test 3

PASS.

In [14]:
def extreme_value_test_3(critical_care_beds=1):
    '''
    Extreme value test 3: 
    
    All unplanned admissions have their inter-arrival time is
    set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model is "Elective"
    '''
    experiment = Experiment(num_critical_care_beds=critical_care_beds)
    run_test(experiment)

In [15]:
extreme_value_test_3()

Patient 1 arrived from Emergency surgery at time 9.310384612032701
Patient 1 admitted to critical care bed at time 9.310384612032701
Elective surgery patient 2 arrived at time 12.802237923916287
Elective surgery for patient 2 cancelled due to no available critical care beds at time 12.802237923916287
Patient 3 arrived from Emergency surgery at time 18.384338881928254
Patient 4 arrived from other hospitals at time 30.31328010044033
Patient 5 arrived from other hospitals at time 34.7254480062633
Patient 6 arrived from Accident and Emergency at time 34.930846627197894
Patient 7 arrived from the Wards at time 35.12542367027909
Elective surgery patient 8 arrived at time 35.450862616151014
Elective surgery for patient 8 cancelled due to no available critical care beds at time 35.450862616151014
Patient 9 arrived from the Wards at time 52.71332286229041
Elective surgery patient 10 arrived at time 53.64451698892128
Elective surgery for patient 10 cancelled due to no available critical care bed

### Test 5: Extreme value test 4

PASS

In [16]:
def extreme_value_test_4(extreme_value=M):
    '''
    Extreme value test 4: 
    
    means of stay_distributions set to $M$ a very large number.
    Expected result: after 24 arrivals queues form and no patients are admitted.
    '''
    mod_stay_distributions=[(M, 267.51), 
                            (M, 276.54), 
                            (M, 218.02), 
                            (M, 457.67), 
                            (M, 108.67), 
                            57.34] 
    experiment = Experiment(stay_distributions=mod_stay_distributions, 
                            elective_treatment_mean=M)
    run_test(experiment)

In [17]:
extreme_value_test_4()

Patient 1 arrived from the Wards at time 0.3271860766010615
Patient 1 admitted to critical care bed at time 0.3271860766010615
Patient 2 arrived from Accident and Emergency at time 2.3562425086122527
Patient 2 admitted to critical care bed at time 2.3562425086122527
Patient 3 arrived from other hospitals at time 4.393683934163507
Patient 3 admitted to critical care bed at time 4.393683934163507
Patient 4 arrived from Emergency surgery at time 8.17522678363743
Patient 4 admitted to critical care bed at time 8.17522678363743
Patient 5 arrived from the Wards at time 12.62035504287709
Patient 5 admitted to critical care bed at time 12.62035504287709
Elective surgery patient 6 arrived at time 17.688164618583063
Elective surgery patient 6 admitted to critical care bed at time 17.688164618583063
Patient 7 arrived from the Wards at time 17.94719546510511
Patient 7 admitted to critical care bed at time 17.94719546510511
Patient 8 arrived from Accident and Emergency at time 38.97932559982171
Pat